# Individual Assignment 3: 
## Crowd Counting: Count the Number of Customers Appeared in the Business

![Imgur](https://i.imgur.com/r1U9dHD.png) 

##**Scenario**

>"We see our customers as invited guests to a party, and we are the hosts. It’s our job every day to make every important aspect of the customer experience a little bit better."   **--- Jeff Bezos, Amazon CEO**



This is also true for all brick-and-motor stores at shopping malls. As a host, one thing you definitely should know is: *How many `guests` are attending your `party`*. 

The #(sign for quantity) of customers are curcial. Key factors for a successful business include `store rent`, `store sales`, which are all strongly correlated with `customers quantity`. \

 

In Practice, keeping track of passenger flow or customer traffic is never an easy thing. At the old times, customers are counted manually. Later, sensors are installed at the entrance, which provide a higher-accuracy solution, but suffers from high cost (Just imagine how many sensors should be installed). \

Not all malls have the luxury of installing sensors, but all malls have video surveillance cameras. With the images captured by real-time cameras, computer vision technology can help counting customers 24/7.




<img src="https://storage.googleapis.com/kaggle-datasets-images/526740/966025/1b806f39569b1157b1aaafe81f59f4b6/dataset-cover.jpg?t=2020-02-24-06-34-25">

In [1]:
#This step is done to connect the content directory to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Task**
Build a neural network to count the number of customers appeared in the image captured so we can understand the potential customers.


---




If you would like to download the dataset, please use below link. 

In [ ]:
link = 'https://drive.google.com/file/d/1RwuuNLpQWcozMOGw6Fp42m_dIhuqH90y/view?usp=sharing' # The shareable link


### Unzip the data file
<font color="orange"><b>Todo:</b></font>\
Replace the `PATH_TO_ZIP_FILE` with the path to your data file in your Google Drive. We are unzipping the data file to the `/content/crowd-counting` directory, which won't use your Google Drive storage.

In [2]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/deep learning/lab4/Crowd-Counting-Dataset.zip" -d /content/crowd-counting # you should use your own link for the zip file

### Load csv files and train-test-split
<font color="orange"><b>Todo:</b></font>\
(1) Read `/content/crowd-counting/labels.csv`: Read csv files as `Pandas DataFrame` into RAM\
(2) Use `Pandas` API, create a new column named `dir`, which stores the image file names. \
(3) Split the whole data into `training dataframe` (0.7) and `testing dataframe` (0.3)\
(4) Split the above `testing dataframe` into `testing dataframe`(0.5) and `validation dataframe` (0.5)\
Hint: Try to think about the relationship between the number above and the "?" parameters below.

In [3]:
import pandas as pd
df=pd.read_csv("/content/crowd-counting/labels.csv")
X="dir"
Y="count"
df[X]="seq_"+df.id.astype(str).str.zfill(6)+".jpg"

In [4]:
df.head()

,id,count,dir
0,1,35,seq_000001.jpg
1,2,41,seq_000002.jpg
2,3,41,seq_000003.jpg
3,4,44,seq_000004.jpg
4,5,41,seq_000005.jpg


In [5]:
from sklearn.model_selection import train_test_split
 
# Train-test-validation split
train,_=train_test_split(df,test_size=0.3)
validation,test=train_test_split(_,test_size=0.5)
print(train.shape,validation.shape,test.shape)

(1400, 3) (300, 3) (300, 3)


In [6]:
from PIL import Image
Image.open("/content/crowd-counting/frames/frames/seq_000001.jpg").mode

'RGB'

**Quick check for your answer:**\
(1400, 3) (300, 3) (300, 3)




### Configure data generator
<font color="orange"><b>Todo:</b></font>

Now, let's configure **3** generators: for training, validation, test respectively, in the code chunk below.
 
\
**Note**: \
(1) Simply rescale input to `1./255`.
You don't need to perform augmentations such as rotation, zooming, and so on. The reason is that the testing images is exactly the same as training images.

(2) Here's a very helpful link about how to use the `flow_from_dataframe` API\
https://keras.io/api/preprocessing/image/

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

target_size = (120, 160)

train_datagen = ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_dataframe(train,
                                              directory='/content/crowd-counting/frames/frames',
                                              target_size=target_size,
                                              class_mode='raw',
                                              x_col=X,
                                              y_col=Y,
                                              shuffle=True)
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_dataframe(validation,
                                          directory='/content/crowd-counting/frames/frames',
                                          target_size=target_size,
                                          class_mode='raw',
                                          x_col=X,
                                          y_col=Y)

test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_dataframe(test,
                                            directory='/content/crowd-counting/frames/frames',
                                            target_size=target_size,
                                            class_mode='raw',
                                            x_col=X,
                                            y_col=Y,

                                            )

Found 1400 validated image filenames.
Found 300 validated image filenames.
Found 300 validated image filenames.


**Quick check:**\
The result should be like: \


Found 1400 validated image filenames.\
Found 300 validated image filenames.\
Found 300 validated image filenames.


### Build Model
<font color="orange"><b>Todo:</b></font>\
Build your convolutional neural network sequential model. 
First, build a 2-hidden layer model. Then build a CNN model with 2 convolution layers. Your CNN model should outperform the dense layer model in order to get a full grade.

#### Dense Layer Model

In [18]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping, Callback
model=Sequential()
model.add(Flatten())  
model.add(Dense(128 ,input_shape = (120*160*3,) ,activation = 'relu'))     
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))


model.compile(loss="mae",                   
              optimizer="adam",
              metrics=['mape'])


#      # EarlyStopping with patience 2: if in 2 consecutive epochs, the validation loss didn't hit the record, training will stop.

# Fit the model and record the training history for plotting purpose
history=model.fit(train_gen,
      steps_per_epoch=20, #train_gen.n//train_gen.batch_size. In lab 2, we set "batchsize" in model.fit() to determine the step length for steps in one epoch. 
                  #Steps_per_epoch is the number of steps in one epoch. It can also determine the step length (batchszie) in each epoch. So, we can set steps_per_epoch instead to replace "batchsize".
                  #Either batchsize or steps_per_epoch is fine. Just make sure you set one of them.
      epochs=20,
      validation_data=val_gen,
      validation_steps=10, #val_gen.n//val_gen.batch_size
      )


Epoch 1/20
20/20 [==============================] - 6s 221ms/step - loss: 8.8889 - mape: 30.0767 - val_loss: 6.2035 - val_mape: 20.7483
Epoch 2/20
20/20 [==============================] - 4s 217ms/step - loss: 6.0977 - mape: 21.0319 - val_loss: 7.0768 - val_mape: 21.4265
Epoch 3/20
20/20 [==============================] - 5s 245ms/step - loss: 6.2133 - mape: 20.9675 - val_loss: 7.0448 - val_mape: 26.9421
Epoch 4/20
20/20 [==============================] - 6s 319ms/step - loss: 6.0488 - mape: 20.6856 - val_loss: 6.1432 - val_mape: 18.7038
Epoch 5/20
20/20 [==============================] - 5s 230ms/step - loss: 5.5529 - mape: 18.4645 - val_loss: 5.5260 - val_mape: 16.8287
Epoch 6/20
20/20 [==============================] - 5s 248ms/step - loss: 5.2511 - mape: 17.5136 - val_loss: 7.2121 - val_mape: 20.9622
Epoch 7/20
20/20 [==============================] - 4s 212ms/step - loss: 5.0170 - mape: 17.0026 - val_loss: 4.3020 - val_mape: 14.0764
Epoch 8/20
20/20 [==============================

The following code is provided for you to calculate model mse.

In [19]:
from sklearn.metrics import mean_absolute_error
y=[]
for i in range(1+ test_gen.n//test_gen.batch_size):
    y.extend(list(test_gen.next()[1]))

predict=model.predict(test_gen)
mean_absolute_error(predict.flatten(), y)

10/10 [==============================] - 2s 215ms/step


3.409426809946696

#### CNN Model

You can use any hyperparameters and improve your results here. 

In [20]:
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import Callback


model=Sequential()
model.add(Conv2D(128, 3,input_shape=(120,160,3),activation="relu"))
model.add(MaxPooling2D(2))
model.add(Conv2D(64, 3,activation="relu"))
model.add(MaxPooling2D(2))
model.add(Flatten())                                                         # Add Dropout layer, randomly drops 50% neurons from propagation.
model.add(Dense(64,activation="relu", kernel_regularizer="l2"))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="relu"))

model.compile(loss="mae",                   
              optimizer="adam",
              metrics=['mape'])


#      # EarlyStopping with patience 2: if in 2 consecutive epochs, the validation loss didn't hit the record, training will stop.

# Fit the model and record the training history for plotting purpose
history=model.fit(train_gen,
      steps_per_epoch=20, #train_gen.n//train_gen.batch_size. In lab 2, we set "batchsize" in model.fit() to determine the step length for steps in one epoch. 
                  #Steps_per_epoch is the number of steps in one epoch. It can also determine the step length (batchszie) in each epoch. So, we can set steps_per_epoch instead to replace "batchsize".
                  #Either batchsize or steps_per_epoch is fine. Just make sure you set one of them.
      epochs=20,
      validation_data=val_gen,
      validation_steps=10, #val_gen.n//val_gen.batch_size
      )


Epoch 1/20
20/20 [==============================] - 14s 329ms/step - loss: 10.7510 - mape: 31.9391 - val_loss: 7.4940 - val_mape: 25.6271
Epoch 2/20
20/20 [==============================] - 6s 288ms/step - loss: 6.9609 - mape: 21.5017 - val_loss: 6.5480 - val_mape: 20.0105
Epoch 3/20
20/20 [==============================] - 5s 231ms/step - loss: 6.6697 - mape: 21.2535 - val_loss: 6.3997 - val_mape: 21.4551
Epoch 4/20
20/20 [==============================] - 6s 310ms/step - loss: 5.9247 - mape: 18.8142 - val_loss: 6.0502 - val_mape: 20.4705
Epoch 5/20
20/20 [==============================] - 5s 233ms/step - loss: 5.8012 - mape: 18.4227 - val_loss: 5.6010 - val_mape: 17.8399
Epoch 6/20
20/20 [==============================] - 5s 264ms/step - loss: 5.0759 - mape: 16.4219 - val_loss: 6.4008 - val_mape: 23.4750
Epoch 7/20
20/20 [==============================] - 5s 228ms/step - loss: 5.1053 - mape: 15.7079 - val_loss: 4.1818 - val_mape: 13.5609
Epoch 8/20
20/20 [============================

In [21]:
from sklearn.metrics import mean_absolute_error
y=[]
for i in range(1+ test_gen.n//test_gen.batch_size):
    y.extend(list(test_gen.next()[1]))

predict=model.predict(test_gen)
mean_absolute_error(predict.flatten(), y)


10/10 [==============================] - 1s 136ms/step


3.029786243438721

### ***Conclusion and Discussion***

#### **Question 1: Please explain which kinds of layer is used in your CNN model. What are the functions of those layers? Please explain and interpret your results.**



###<font color = blue> The model has these layers:<br>Conv2D layer: This is a convolutional layer that convolves the input image with 3x3 kernels and applies a ReLU activation function to the output for non-linear transformation. The layer has 128 output channels and produces an output size of (118, 158, 128). <br><br>MaxPooling2D layer: This is a pooling layer that downsamples the output of the Conv2D layer to reduce the spatial size of the feature maps, as well as the number of computations and parameters. The layer applies a 2x2 max pooling operation, taking the maximum value from each 2x2 region as output. The layer produces an output size of (59, 79, 128).<br><br>Conv2D layer: This is the second convolutional layer that convolves the output of the previous layer with 3x3 kernels and applies a ReLU activation function to the output for non-linear transformation. The layer has 64 output channels and produces an output size of (57, 77, 64). <br><br>MaxPooling2D layer: This is the second pooling layer that further downsamples the output of the second convolutional layer. The layer applies a 2x2 max pooling operation and produces an output size of (28, 38, 64).<br><br>Flatten layer: This is a flattening layer that flattens the output of the previous layer into a one-dimensional array for subsequent fully connected layers. The layer produces an output size of 68416.<br><br>Dense layer: This is a fully connected layer that maps the input vector to a hidden layer of size 64, applies a ReLU activation function to the output for non-linear transformation, and uses L2 regularization to constrain the model complexity and improve its generalization performance.<br><br>Dense layer: This is the second fully connected layer that maps the output of the previous layer to a hidden layer of size 16 and applies a ReLU activation function to the output for non-linear transformation.<br><br>Dense layer: This is the output layer that maps the output of the previous layer to a single node of size 1 and applies a ReLU activation function as the output activation function. The output of the layer represents the predicted number of people.

#### **Question 2: Which type of error measure did you use for the model training? How does this kind of error measure defined?** 



###<font color = blue> I used mean absolute error (MAE) as the error measure for model training. MAE is a regression loss function that measures the average absolute difference between the predicted values and the actual values. It calculates the mean of the absolute differences between the predicted and actual values, without considering their direction. In other words, MAE represents the average amount by which the predictions differ from the actual values.

#### **Question 3: What business problem can be addressed by CNN? Can you give an example?**

###<font color = blue>One business problem that can be addressed by CNNs is automatic image recognition and classification. For example, a retail company might want to use CNNs to automatically classify product images into different categories, such as clothing, electronics, or home appliances. By automating this process, the company can save time and resources that would otherwise be spent on manual classification. <br><br>Another example is in the healthcare industry, where CNNs can be used to analyze medical images, such as X-rays, MRIs, and CT scans, to identify and diagnose various diseases, such as cancer, heart disease, and neurological disorders. By using CNNs to automate the analysis process, healthcare professionals can save time and increase the accuracy of their diagnoses